In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.values, wine.target, stratify=wine.target, test_size=0.2, random_state=0)

In [4]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

print(X_train.shape)
print(y_train.shape)

torch.Size([142, 13])
torch.Size([142])


In [5]:
train = TensorDataset(X_train, y_train)

print(train[0])

# minibiatch
train_loader = DataLoader(train, batch_size=16, shuffle=True)

(tensor([1.1840e+01, 8.9000e-01, 2.5800e+00, 1.8000e+01, 9.4000e+01, 2.2000e+00,
        2.2100e+00, 2.2000e-01, 2.3500e+00, 3.0500e+00, 7.9000e-01, 3.0800e+00,
        5.2000e+02]), tensor(1))


In [6]:
# 신경망
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 3)
        
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [7]:
model = MyNet()

In [8]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(600):
    total_loss = 0
    for batch_x, batch_y in train_loader:
#         batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    if (epoch+1) % 50 == 0:
        print(epoch+1, total_loss)

50 4.745560854673386
100 4.514144599437714
150 4.772169679403305
200 4.9432094395160675
250 5.740111321210861
300 5.767281740903854
350 5.049948126077652
400 4.516259163618088
450 4.975223749876022
500 6.556899935007095
550 4.752487391233444
600 5.2156510800123215


In [18]:
X_test, y_test = Variable(X_test), Variable(y_test)
result = torch.max(model(X_test).data, 1)[1]

In [19]:
y_test.data.numpy() == result.numpy()

array([ True,  True, False, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True, False, False,  True, False, False,  True, False,
        True,  True, False,  True,  True,  True,  True,  True, False])

In [20]:
accuracy = sum(y_test.data.numpy() == result.numpy()) / len(y_test.data.numpy())
accuracy

0.6666666666666666